## This scripts tests the accuarcy of the models

#### Imort dependencies & load test data into memory:

In [1]:
import mat73
import numpy as np
import math
from scipy.stats import norm, multivariate_normal
from naive_bayes import NaiveBayes
from TAN_model import Model
from probabilities import pClass

In [2]:
# WARNING: this takes about 30s
test_data = mat73.loadmat('preprocessed_data/all_subjects/test.mat')

In [3]:
# WARNING: this takes about 3m 30s
train_data = mat73.loadmat('preprocessed_data/all_subjects/train.mat')

In [4]:
# For testing the accuracy when only one subject is used 
test_data_one_sample = mat73.loadmat('preprocessed_data/one_subject/test.mat')
train_data_one_sample = mat73.loadmat('preprocessed_data/one_subject/train.mat')

#### Test accuracy for predicting the "visible" class using TAN

In [5]:
# Load parameters
CLASS = 'visible'
cov = np.load(('distributions/' + CLASS + '/cov.npy'))
mu_bar = np.load(('distributions/' + CLASS + '/mu_bar.npy'))
single_normal = np.load(('distributions/' + CLASS + '/single_normal.npy'))
parents = np.load(('distributions/' + CLASS + '/parents.npy'))
parents = parents.astype(int)

In [6]:
# Prepare the test data
test_data_dict = test_data['test']
test_X = test_data_dict['data']
test_X = np.array(test_X)[0:1000]
test_y = test_data_dict[CLASS]
test_y = np.array(test_y)[0:1000]

train_data_dict = train_data['train']
train_y = train_data_dict[CLASS]

# Calculate Priors
prior_1 = np.sum(train_y) / len(train_y)
prior_0 = 1 - prior_1
priors_state = [prior_0, prior_1]
class_list = np.unique(test_y).astype(int)

# Run the model
model = Model(single_normal, mu_bar, cov, parents, priors_state)
model.run_test(test_X, test_y)


Running Accuracy: 0.54 windowed: 0.54 for 100 samples
Running Accuracy: 0.565 windowed: 0.59 for 200 samples
Running Accuracy: 0.57 windowed: 0.58 for 300 samples
Running Accuracy: 0.5675 windowed: 0.56 for 400 samples
Running Accuracy: 0.564 windowed: 0.55 for 500 samples
Running Accuracy: 0.5783333333333334 windowed: 0.65 for 600 samples
Running Accuracy: 0.5614285714285714 windowed: 0.46 for 700 samples
Running Accuracy: 0.56375 windowed: 0.58 for 800 samples
Running Accuracy: 0.5644444444444444 windowed: 0.57 for 900 samples
Accuracy is 0.571


In [17]:
class NaiveBayes2():

    def __init__(self, single_normal, num_classes=2, num_dims=300):
        self.single_normal = single_normal
        self.num_classes = num_classes
        self.num_dims = num_dims

    def classify(self, x):
        x_given_class = np.zeros((self.num_dims,))

        for c in range(self.num_classes):
            mus = self.single_normal[:, c, 0]
            sigmas = self.single_normal[:, c, 1]
            node_probs = norm.pdf(x, mus, sigmas)
            x_given_class[c] = np.prod(node_probs)

        return np.argmax(x_given_class)

    def classi(self, x):
        node_probs = norm.pdf(
            x, 
            self.single_normal[:, :, 0], 
            self.single_normal[:, :, 1]
        )
        x_given_class = np.prod(node_probs, axis=0)
        return np.argmax(x_given_class)

    def test(self, X, y):
        N, _ = X.shape
        correct = 0

        for i in range(N):
            x = X[i]
            prediction = self.classify(x)
            if prediction == y[i]:
                correct += 1

            if i % 100 == 0 and i != 0:
                print(f'Running Accuracy is {correct / i} for {i} samples')

#### Test accuracy for predicting the "visible" class using Naive Bayes

In [18]:
# Run the model
nb = NaiveBayes2(single_normal)
nb.test(test_X, test_y)

ValueError: maximum supported dimension for an ndarray is 32, found 300

#### Test accuracy for predicting the "visible" class using an intuitive MST

In [ ]:
# Load parameters
root = 'distributions/' + CLASS + '/intuitive/'
cov_state_intuit = np.load((root + 'cov.npy'))
mu_bar_state_intuit = np.load((root + 'mu_bar.npy'))
single_normal_state_intuit = np.load((root + 'single_normal.npy'))
parents_intuit = np.arange(-1, 299)

In [ ]:
# Prepare test data
test_data_dict = test_data['test']
test_X = test_data_dict['data']
test_X = np.array(test_X)[0:1000]
test_y_vis = test_data_dict['visible']
test_y_vis = np.array(test_y_vis)[0:1000]

train_data_dict = train_data['train']
train_y_vis = train_data_dict['visible']
prior_1 = np.sum(train_y_vis) / len(train_y_vis)
prior_0 = 1 - prior_1
priors_state = [prior_0, prior_1]

In [ ]:
# Run model
model = Model(single_normal_state_intuit, mu_bar_state_intuit, cov_state_intuit, parents_intuit, priors_state)
model.run_test(test_X, test_y)

#### Test accuracy for predicting the "position" class using TAN

In [ ]:
# Load parameters
CLASS = 'position'
root = 'distributions/' + CLASS + '/'
cov = np.load((root + 'cov.npy'))
mu_bar = np.load((root + 'mu_bar.npy'))
single_normal = np.load((root + 'single_normal.npy'))
parents = np.load((root + '/parents.npy'))
parents = parents.astype(int)

In [ ]:
# Prepare the test data
test_data_dict = test_data['test']
test_X = test_data_dict['data']
test_X = np.array(test_X)[0:1000]
test_y = test_data_dict[CLASS]
test_y = np.array(test_y)[0:1000]

train_data_dict = train_data['train']
train_y = train_data_dict[CLASS]

# Calculate Priors
class_list = np.unique(test_y).astype(int)
priors_state = np.zeros((len(class_list),1))
for i in class_list:
    priors_state[i-1] = pClass(train_y, i)

# Run the model
model = Model(single_normal, mu_bar, cov, parents, priors_state, class_list)
model.run_test(test_X, test_y)

#### Test accuracy for predicting the "position" class using Naive Bayes